## Cargar Datos

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
from pyspark.sql.functions import col, lit, current_timestamp

train_df = spark.read.table("laboratorio3.features.train_df").toPandas()

# RECONSTRUIR DATASET DE INFERENCIA (JOIN)
predictions_df = spark.read.table("laboratorio3.results.predictions")
features_inferencia_df = spark.read.table("laboratorio3.features.features_inferencia")

inference_full_df = predictions_df.join(
    features_inferencia_df, 
    on="Transaction_ID", 
    how="inner"
).filter("inference_timestamp >= date_sub(current_timestamp(), 7)") # Últimos 7 días

prod_df = inference_full_df.toPandas()


In [0]:
# 3. FUNCIÓN DE CÁLCULO PSI (Population Stability Index)
def calculate_psi(expected, actual, buckets=10):
    breakpoints = np.arange(0, buckets + 1) / (buckets) * 100
    breakpoints = np.percentile(expected, breakpoints)
    breakpoints[0] = -np.inf
    breakpoints[-1] = np.inf

    expected_percents = np.histogram(expected, bins=breakpoints)[0] / len(expected)
    actual_percents = np.histogram(actual, bins=breakpoints)[0] / len(actual)

    # Evitar ceros
    actual_percents = np.clip(actual_percents, 0.0001, 1) 
    expected_percents = np.clip(expected_percents, 0.0001, 1)

    psi_value = np.sum((expected_percents - actual_percents) * np.log(expected_percents / actual_percents))
    return psi_value

# 4. COMPARAR FEATURES (Drift de entrada)
features_to_monitor = ['Transaction_Amount', 'Account_Balance']
results = []

print("--- Análisis de Data Drift ---")
for feature in features_to_monitor:
    psi = calculate_psi(train_df[feature], prod_df[feature])
    status = "🚨 DRIFT ALTO" if psi > 0.2 else ("⚠️ CAMBIO LEVE" if psi > 0.1 else "✅ ESTABLE")
    
    results.append((feature, float(psi), status))  # Cast psi to float
    print(f"Feature: {feature:20} | PSI: {psi:.4f} | Estado: {status}")

# 5. GUARDAR RESULTADOS PARA PROMETHEUS / GRAFANA
drift_history_df = spark.createDataFrame(results, ["feature", "psi_score", "status"]) \
    .withColumn("monitored_at", current_timestamp())

drift_history_df.write.format("delta").mode("append").saveAsTable("laboratorio3.results.monitoring_drift_history")